In [1]:
import pandas as pd

node_df = pd.read_csv('MOFid-v5_node.csv', header=0)

node_df

,Unnamed: 0,MOF,Solv_remove,Formula,Type,Node-file
0,8035,10876_2018_1483_MOESM2_ESM,ASR,C6Cu3O12,Type-1,10876_2018_1483_MOESM2_ESM_ASR_pacman_node0
1,8034,10876_2018_1483_MOESM2_ESM,FSR,C6Cu3H8O16,Type-1,10876_2018_1483_MOESM2_ESM_FSR_pacman_node0
2,12171,10876_2020_1817_MOESM2_ESM,ASR,Cu1,Type-1,10876_2020_1817_MOESM2_ESM_ASR_pacman_node0
3,12172,41467_2015_BFncomms7350_MOESM943_ESM_1,ASR,Cu1,Type-1,41467_2015_BFncomms7350_MOESM943_ESM_1_ASR_pac...
4,12173,41467_2015_BFncomms7350_MOESM943_ESM_1,FSR,Cu1,Type-1,41467_2015_BFncomms7350_MOESM943_ESM_1_FSR_pac...
...,...,...,...,...,...,...
18622,9981,sc9b00062_si_002,FSR,C8H4O24Zr6,Type-1,sc9b00062_si_002_FSR_pacman_node0
18623,5835,science.1056698,ASR,C4Cu2O8,Type-1,science.1056698_ASR_pacman_node0
18624,3756,tz9b00021_si_002,ASR,C36H24N24Ni8O6,Type-1,tz9b00021_si_002_ASR_pacman_node0
18625,3757,tz9b00021_si_002,FSR,C36H24N24Ni8O6,Type-1,tz9b00021_si_002_FSR_pacman_node0


In [2]:
node_dict = {}

with open('MOFid-v5_node.csv', 'r') as f:
    for index, line in enumerate(f):
        if index >= 1:
            data = line.strip().split(',')
            mofname = data[1] + '_' + data[2] + '_pacman'
            nodename = f'[{data[3]}_{data[4]}]'
            if mofname not in node_dict:
                node_dict[mofname] = []
                node_dict[mofname].append(nodename)
            else:
                node_dict[mofname].append(nodename)


In [3]:
from ase.io import read as ase_read
from ase.io import write as ase_write
import networkx as nx
import selfies as sf

def dict2str(dct):
    """Convert symbol-to-number dict to str.
    """
    return ''.join(symb + (str(n)) for symb, n in dct.items())

metals  = ['Li', 'Na', 'K', 'Rb', 'Cs', 'Fr', 'Be', 'Mg', 'Ca', 'Sr', 'Ba', 'Ra',
          'Al', 'Ga', 'Ge', 'In', 'Sn', 'Sb', 'Tl', 'Pb', 'Bi', 'Po',
          'Sc', 'Ti', 'V' , 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn',
          'Y' , 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd',
          'Hf', 'Ta', 'W' , 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg',
          'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'U', 'Tm', 'Yb', 'Lu',
          'Ac', 'Th', 'Pa', 'U', 'Np', 'Pu', 'Am', 'Cm', 'Bk', 'Cf', 'Es', 'Fm', 'Md', 'No', 'Lr'
         ]


In [7]:
# Convert SMILES string to SELFIES string

test_mofid = '[Ni].[O]P(=O)(Cc1ccc(cc1)C1=NC(=[N]=C([N]1)c1ccc(cc1)CP(=O)(O)O)c1ccc(cc1)CP(=O)(O)[O])O.[O]P(=O)(Cc1ccc(cc1)C1=NC(=[N]=C([N]1)c1ccc(cc1)CP(=O)(O)O)c1ccc(cc1)CP(=O)([O])O)O MOFid-v1.ERROR.cat0;ABAVIM01_ASR'
def convert_mofid_to_mofidv2(mofid):
    def join_mofidv2(selfies, node_type, v1_details):
        v2_details = v1_details.replace("v1", "v2")
        
        if selfies == '*' and node_type == '*':
            return '*' + ' ' + v2_details
        else:
            return node_type + '.' + selfies + ' ' + v2_details
    
    def contain_metal(smile):
        check = False
        for m in metals:
            if m in smile:
                check = True
        return check
    
    def smiles_to_selfies(smiles):    
        selfies = []
        for smile in smiles:
            if contain_metal(smile):
                continue
            else:
                try:
                    selfie = sf.encoder(smile)
                except:
                    selfie = 'ERROR'
                selfies.append(selfie)
        return '.'.join(selfies)

    smiles = mofid.split()[0].split('.')
    v1_details = mofid.split()[1]

    if smiles == ['*']:
        selfies = '*'
        node_type = '*'
    else:
        selfies = smiles_to_selfies(smiles)
        mofname = v1_details.split(';')[-1]
        try:
            nodes = node_dict[mofname]
        except KeyError:
            print(mofname)
            nodes = ['*']
        if len(nodes) == 1:
            node_type = nodes[0]
        else:
            node_type = '.'.join(nodes)
        
    mofid_v2 = join_mofidv2(selfies, node_type, v1_details)
    return mofid_v2
convert_mofid_to_mofidv2(test_mofid)                
    

ABAVIM01_ASR


'*.ERROR.ERROR MOFid-v2.ERROR.cat0;ABAVIM01_ASR'

In [9]:
# ASR
mofids = []
mofids_v2 = []
with open('ASR_mofid-v1.txt', 'r') as f:
    for line in f:
        mofids.append(line.strip())
with open('ASR_mofid-v2.txt', 'w') as f:        
    for idx, mofid in enumerate(mofids):
        mofid_v2 = convert_mofid_to_mofidv2(mofid)
        f.write('{}\n'.format(mofid_v2))

RUCBID_ASR_pacman
RAYFAA_ASR_pacman
GACVIS_ASR_pacman
GIWYET_ASR_pacman
FEYPUX_ASR_pacman
PIDPIE_ASR_pacman
d1ra04824d2_ASR_pacman
VUCHIN_ASR_pacman
VEHBUI_ASR_pacman
VEHBOC_ASR_pacman
c9ce01675a2_3_ASR_pacman
c7ce01840a2_3_ASR_pacman
c7dt01045a2_1_ASR_pacman
c7ra08188j2_3_ASR_pacman
CENJIQ_ASR_pacman
CENKAJ_ASR_pacman
UKAQED01_ASR_pacman
UKAQED_ASR_pacman
UKAQAZ_ASR_pacman
UKAPUS_ASR_pacman
UKAQIH_ASR_pacman
JUVMIZ_ASR_pacman
JUGYAO_ASR_pacman
BIKNUH_ASR_pacman
tz9b00021_si_006_ASR_pacman
IRIMOO02_ASR_pacman
IRIMOO01_ASR_pacman
SIMLIN_ASR_pacman
SODKON_ASR_pacman
SIWTEZ_ASR_pacman
ZIGQEO_ASR_pacman
ZIGQAK_ASR_pacman


In [10]:
# FSR
mofids = []
mofids_v2 = []
with open('FSR_mofid-v1.txt', 'r') as f:
    for line in f:
        mofids.append(line.strip())
with open('FSR_mofid-v2.txt', 'w') as f:        
    for idx, mofid in enumerate(mofids):
        mofid_v2 = convert_mofid_to_mofidv2(mofid)
        f.write('{}\n'.format(mofid_v2))

RAYFAA_FSR_pacman
RUCBID_FSR_pacman
GACVIS_FSR_pacman
FEYPUX_FSR_pacman
PIDPIE_FSR_pacman
d1ra04824d2_FSR_pacman
VEHBOC_FSR_pacman
VEHBUI_FSR_pacman
c9ce01675a2_3_FSR_pacman
c7ra08188j2_3_FSR_pacman
c7ce01840a2_3_FSR_pacman
c7dt01045a2_1_FSR_pacman
CENJIQ_FSR_pacman
UKAQIH_FSR_pacman
UKAQED01_FSR_pacman
UKAPUS_FSR_pacman
UKAQED_FSR_pacman
UKAQAZ_FSR_pacman
JUVMIZ_FSR_pacman
BIKNUH_FSR_pacman
tz9b00021_si_006_FSR_pacman
IRIMOO02_FSR_pacman
IRIMOO01_FSR_pacman
SIWTEZ_FSR_pacman
SODKON_FSR_pacman
SIMLIN_FSR_pacman
ZIGQEO_FSR_pacman


In [11]:
# Ions
mofids = []
mofids_v2 = []
with open('ion_mofid-v1.txt', 'r') as f:
    for line in f:
        mofids.append(line.strip())
with open('ion_mofid-v2.txt', 'w') as f:        
    for idx, mofid in enumerate(mofids):
        mofid_v2 = convert_mofid_to_mofidv2(mofid)
        f.write('{}\n'.format(mofid_v2))

ic501477u_si_003_ion_pacman


In [13]:
refs1 = []
refs2 = []
linkers = []
topos = []
cats = []
metal_nodes = []
metals  = ['Li', 'Na', 'K', 'Rb', 'Cs', 'Fr', 'Be', 'Mg', 'Ca', 'Sr', 'Ba', 'Ra',
          'Al', 'Ga', 'Ge', 'In', 'Sn', 'Sb', 'Tl', 'Pb', 'Bi', 'Po',
          'Sc', 'Ti', 'V' , 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn',
          'Y' , 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd',
          'Hf', 'Ta', 'W' , 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg',
          'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'U', 'Tm', 'Yb', 'Lu',
          'Ac', 'Th', 'Pa', 'U', 'Np', 'Pu', 'Am', 'Cm', 'Bk', 'Cf', 'Es', 'Fm', 'Md', 'No', 'Lr'
         ]

def contain_metal(smile):
    check = False
    for m in metals:
        if m in smile:
            check = True
    return check

with open('all_mofid-v1.txt', 'r') as f:
    for line in f:
        if line.startswith("*"):
            continue
        else:
            data = line.strip().split(';')
            refs1.append(data[1])
            v1 = data[0].split()
            big_smile = v1[0].split('.')
            others = v1[1]
            
            final_smile = []
            others = others.split('.')

            for smile in big_smile:
                if contain_metal(smile):
                    continue
                else:
                    final_smile.append(smile)
            linker = '.'.join(final_smile) #Linker SMILES
            topo = others[1]
            cat = others[-1]
            topos.append(topo)
            cats.append(cat)
            linkers.append(linker)
            
with open('all_mofid-v2.txt', 'r') as f:
    for line in f:
        if line.startswith("*"):
            continue
        else:
            data = line.strip().split(';')
            refs2.append(data[1])
            v2 = data[0].split()
            big_smile = v2[0].split('.')
            others = v2[1]
            
            final_smile = []

            for smile in big_smile:
                if contain_metal(smile):
                    final_smile.append(smile)
            metal_node = '.'.join(final_smile) # Metal node
            print(metal_node)
            metal_nodes.append(metal_node)
with open('data.csv', 'w') as f:
    for i, j, k, l in zip(refs1, linkers, topos, cats):
        try:
            id_ref2 = refs2.index(i)
            #print(id_ref2)
            metal_node = metal_nodes[id_ref2]
        except ValueError:
            metal_node = None
        f.write(f"{i}|{metal_node}|{j}|{k}|{l}\n")


[C4In1O8_Type-1]
[Cd1_Type-1]
[Cd1_Type-1]
[Br1Zn1_Type-1]
[C6O14Zn4_Type-1]
[C16H12N18O8Zn5_Type-1]
[C16H12N18O8Zn5_Type-2]
[C4In1O8_Type-1].[C8O16Tb2_Type-1]
[C4Co2O8_Type-1]
[C2Co1O4_Type-2]
[Ce1_Type-1]
[Pr1_Type-1]
[La1_Type-1]
[Cu1_Type-1]
[C4Cu2O8_Type-1]
[C2O4Zn1_Type-3]
[C3Cd2O6_Type-1]
[C4Cd2O8_Type-1]
[C4Cd2O8_Type-1]
[C2Ni1O4_Type-1]
[C4O8Zn2_Type-1]
[C6H4In3O16_Type-1]
[C6Cd3O12_Type-1].[Cd1_Type-1]
[C6Co3O12_Type-1]
[C4O8Zn2_Type-1]
[C4O8Zn2_Type-1]
[C2O4Zn1_Type-1]
[C4Cd2O8_Type-1]
[C4Cd2O8_Type-1]
[C6Fe3O13_Type-1]
[C6Co3O13_Type-1]
[C4Co2O8_Type-3]
[C2Cd1O4_Type-1]
[C9O26Zr6_Type-1]
[C9O26Zr6_Type-1]
[C4O8Zn2_Type-1]
[In1_Type-1]
[Mg3_Type-1]
[Mg6_Type-2]
[Mg3_Type-1]
[Co3_Type-1]
[Co3_Type-1]
[Ni3_Type-1]
[Ni6_Type-1]
[Ni3_Type-1]
[C4La2O8_Type-1]
[C4Nd2O8_Type-2]
[C4Lu2O8_Type-1]
[C4La2O8_Type-1]
[C4Dy2O8_Type-1]
[C4Lu2O8_Type-1]
[C4Nd2O8_Type-2]
[C4Co2O9_Type-1]
[C4Co2O8_Type-1]
[C6Fe3O13_Type-1]
[Zr2_Type-1]
[Zn1_Type-1]
[Zn1_Type-1]
[La2_Type-1]
[Ce2_Type-1]
[Nd2_

[C12O39Zr9_Type-1]
[Mg1_Type-1]
[Cu1_Type-1]
[Cu1_Type-1]
[Cu1_Type-1]
[Fe1_Type-1]
[Fe1_Type-1]
[Zn1_Type-1]
[Zn1_Type-1]
[C4Cu2O8_Type-1]
[C4Cu2O8_Type-1]
[C24H16N16Ni4_Type-1]
[V4_Type-1]
[V4_Type-1]
[V4_Type-1]
[C8O24Zr6_Type-1]
[C8Hf6O24_Type-1]
[Co1_Type-1]
[Co1_Type-1]
[Co1_Type-1]
[Co1_Type-1]
[Co1_Type-1]
[Co1_Type-1]
[Co1_Type-1]
[Co1_Type-1]
[Co1_Type-1]
[Zn1_Type-1]
[Zn1_Type-1]
[C4O8Zn2_Type-2]
[Zn1_Type-1]
[Co1_Type-1]
[Co1_Type-1]
[C12H4O32Zr6_Type-5]
[C4O8Zn2_Type-1]
[Zn1_Type-1]
[Cr1_Type-1].[Mn1_Type-1]
[Cd1_Type-1]
[Zn1_Type-1]
[Zn1_Type-1]
[Zn1_Type-1]
[Zn1_Type-1]
[C4O8Zn2_Type-1]
[C6Cd2O12_Type-1]
[C2O4Zn1_Type-1].[C2O5Zn2_Type-1]
[C4O8Zn2_Type-1].[C6Cr3O13_Type-1]
[C4O8Zn2_Type-1].[C6Cr3O13_Type-1]
[C4O8Zn2_Type-1].[C6Cr3O13_Type-1]
[C4O8Zn2_Type-1].[C6Cr3O13_Type-1]
[C4O8Zn2_Type-1].[C6Cr3O13_Type-1]
[C4O8Zn2_Type-1].[C6Cr3O13_Type-1]
[C4O8Zn2_Type-1].[C6Cr3O13_Type-1]
[C12H4O32Zr6_Type-3]
[Zn1_Type-1]
[C2Cu1O4_Type-1]
[C2Cu1O4_Type-1]
[C2Cu1O4_Type-1]
[C2Cu1O4_

[C6Fe3O13_Type-1]
[C12Fe6H6O32_Type-1]
[C6Fe3O13_Type-1]
[C6Fe3O13_Type-1]
[C6Fe3H3O16_Type-1]
[C6Fe3O13_Type-1]
[C6Fe3O13_Type-1]
[C6Fe3O13_Type-1]
[Zn1_Type-1]
[C4Fe1O8_Type-1].[Zn1_Type-1]
[Al1_Type-1]
[Al1_Type-1]
[Al1_Type-1]
[Al1_Type-1]
[C4Co2O8_Type-1]
[C6Gd2O12_Type-1]
[C6Eu2O12_Type-1]
[C6Gd2O12_Type-1]
[C6Gd2O12_Type-1]
[C6Gd2O12_Type-1]
[Ti12_Type-1]
[Ti12_Type-1]
[Ti12_Type-1]
[C6Co3O12_Type-1].[C6O13Ru3_Type-1]
[Bi1C4O8_Type-1]
[Cd1_Type-1]
[C4Cd2O8_Type-1]
[Zn1_Type-1]
[Zn1_Type-1]
[Zn1_Type-1]
[Zn1_Type-1]
[Zn1_Type-1]
[Zn1_Type-1]
[Zn1_Type-1]
[Zn1_Type-1]
[C2Ni1O4_Type-3]
[C2Ni1O4_Type-3]
[C4Cd2O8_Type-1]
[Ni1_Type-1]
[C4Cu2O8_Type-1]
[C4Cu2O8_Type-1]
[C4Cu2O8_Type-1]
[C4Cu2O8_Type-1]
[C4Cu2O8_Type-1]
[C4Cu2O8_Type-1]
[C4Cu2O8_Type-1]
[C4Cu2O8_Type-1]
[C4Cu2O8_Type-1]
[C4Cu2O8_Type-1]
[C4Cu2O8_Type-1]
[C4Cu2O8_Type-1]
[C4Cu2O8_Type-1]
[C4Cu2O8_Type-1]
[C4Cu2O8_Type-1]
[C4Cu2O8_Type-2]
[C12Cd3Eu2N16O16_Type-1]
[C2Cd1O4_Type-1]
[C2Cd1O4_Type-1]
[Ag2_Type-1]
[C10H6N6O8Zn